In [1]:
import os

from ConST.prepare_data.data_utils import load_df_from_tsv, save_df_to_tsv

In [10]:
data_root = '/mnt/data/siqiouyang/datasets/must-c-v1.0'
# filenames = ['train-0h_asr', 'train-1h_asr_100h', 'train-1h_asr_10h']
# new_filenames = ['train_pseudo_st_370h', 'train_pseudo_st_100h', 'train_pseudo_st_10h']
filenames = ['train-1h_asr_10h']
new_filenames = ['train_pseudo_0.1mt_st_10h']

In [11]:
os.makedirs('generations', exist_ok=True)

```bash
CUDA_VISIBLE_DEVICES=1 fairseq-generate /mnt/data/siqiouyang/datasets/must-c-v1.0/ --gen-subset train-1h_asr_10h --task speech_to_text --prefix-size 1 \
--max-tokens 4000000 --max-source-positions 4000000 --beam 10 --lenpen 0.6 --scoring sacrebleu \
--config-yaml config_st_de.yaml  --path /mnt/data/siqiouyang/runs/ConST/pretrained/checkpoint.0.1wmt.best_bleu_avg.pt \
--results-path /home/siqiouyang/work/projects/ConST/ConST/prepare_data/generations/train-1h_asr_10h_0.1mt \
--mt-mode 
```

In [5]:
subset = 0

In [6]:
fn = filenames[subset]
df = load_df_from_tsv(os.path.join(data_root, fn + '.tsv'))

In [12]:
# gen_fn = 'generations/{0}/generate-{0}.txt'.format(fn)
gen_fn = 'generations/train-1h_asr_10h_0.1mt/generate-train-1h_asr_10h.txt'

In [13]:
def extract(path):
    gens = {}
    with open(path, 'r') as r:
        for line in r.readlines():
            line = line.strip()
            parts = line.split('\t')
            if line.startswith('D-'):
                idx = int(line[2:line.find('\t')])
                gens[idx] = parts[2]
    return gens

In [14]:
gens = extract(gen_fn)

In [15]:
_, pseudo_labels = list(zip(*sorted(gens.items())))

In [16]:
new_df = df.copy()
new_df['tgt_text'] = pseudo_labels

In [17]:
new_fn = new_filenames[subset]
save_df_to_tsv(new_df, os.path.join(data_root, new_fn + '.tsv'))